# Shortest Path Tutorial

![mathematical model](new_model.png)

![Route](problem.png)

In [16]:
from gurobipy import *
import pandas as pd 
import numpy as np 

model = Model("Shortest path problem")


I = [i for i in range(5)]
J = [i for i in range(5)]

origin = 0
dest = 4

d = {(0,0):0,(0,1):5,(0,2):8,(0,3):0,(0,4):0,
     (1,0):0,(1,1):0,(1,2):0,(1,3):2,(1,4):0,
     (2,0):0,(2,1):8,(2,2):0,(2,3):0,(2,4):4,
     (3,0):0,(3,1):0,(3,2):3,(3,3):0,(3,4):3,
     (4,0):0,(4,1):0,(4,2):0,(4,3):0,(4,4):0,
    }


x = {}

for key in d.keys():
    x[key[0],key[1]] = model.addVar(vtype="B", name="x(%s,%s)"%(key[0],key[1]))



model.addConstr(quicksum(x[key[0],key[1]] for key in d.keys() if key[0] == origin) == 1, name="start")
model.addConstr(quicksum(x[key[0],key[1]] for key in d.keys() if key[1] == dest) == 1, name="end")

for key in d.keys():
    model.addConstr(d[key[0],key[1]] * x[key[0],key[1]] >= 0  , name = "non-zero")

for j in J:
    if j != origin and j != dest:
        model.addConstr(quicksum(x[i,j] for i in I) - quicksum(x[j,k] for k in I) == 0, name="flow(%s)"%j)



model.setObjective(quicksum(d[key[0],key[1]]*x[key[0],key[1]] for key in d.keys()),GRB.MINIMIZE)

model.optimize()
model.write('test.lp')


    
    

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 5 rows, 25 columns and 34 nonzeros
Model fingerprint: 0x317d442a
Variable types: 0 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


In [15]:
for var in model.getVars(): 
    if(var.x > 0):
        print(var.varName, '\t', var.x) 


x(0,0) 	 1.0
x(1,1) 	 1.0
x(2,2) 	 1.0
x(3,3) 	 1.0
x(4,0) 	 1.0
x(4,4) 	 1.0


In [112]:
from gurobipy import *
import pandas as pd 
import numpy as np 

Nodes = ['s', 'a', 'b', 'c', 't'] 

Arcs = {('s','a'): 5 
        ,('s','b'): 8
        ,('a','c'): 2
        ,('b','a'): -10
        ,('c','b'): 3
        ,('b','t'): 4
        ,('c','t'): 3
       }


model = Model('dual problem') 

# add decision variables 
X = {}
for key in Arcs.keys():
    index = 'x_' + key[0] + ',' + key[1] 
    X[key] = model.addVar(vtype=GRB.BINARY 
                          , name= index 
                         ) 

# add objective function
obj = LinExpr(0) 
for key in Arcs.keys():
    obj.addTerms(Arcs[key], X[key])

model.setObjective(obj, GRB.MINIMIZE) 

# constraint1 1 and constraint 2  
lhs_1 = LinExpr(0)
lhs_2 = LinExpr(0)
for key in Arcs.keys():
    if(key[0] == 's'):
        lhs_1.addTerms(1, X[key])
    elif(key[1] == 't'):
        lhs_2.addTerms(1, X[key])
model.addConstr(lhs_1 == 1, name = 'start flow')
model.addConstr(lhs_2 == 1, name = 'end flow') 

# constraints 3
for node in Nodes:
    lhs = LinExpr(0)
    if(node != 's' and node != 't'):
        for key in Arcs.keys():
            if(key[1] == node):
                lhs.addTerms(1, X[key])
            elif(key[0] == node):
                lhs.addTerms(-1, X[key])
    model.addConstr(lhs == 0, name = 'flow conservation')  

model.write('model_spp.lp')
model.optimize()
 
print(model.ObjVal) 
for var in model.getVars(): 
    if(var.x > 0):
        print(var.varName, '\t', var.x) 


Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 7 rows, 7 columns and 14 nonzeros
Model fingerprint: 0x48f1b1d2
Variable types: 0 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 7 rows and 7 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%
3.0
x_s,b 	 1.0
x_a,c 	 1.0
x_b,a 	 1.0
x_c,t 	 1.0


x_s,b 	 1.0
x_a,c 	 1.0
x_b,a 	 1.0
x_c,t 	 1.0
